RCP_TTL: 레시피 제목<br>
RCMM_CNT: 추천 횟수 <br>
SRAP_CNT: 스크랩 횟수 <br>
CKG_MTH: 요리 방법 <br>
CKG_STA_ACTO_NM: 요리 목적 <br>
CKG_MTRL_ACTO_NM: 주재료의 종류<br>
CKG_KND: 요리 종류 <br>
CKG_IPDC: 요리 소개<br>
CKG_MTRL_CN: 요리 상세 재료<br>
CKG_INBUN_NM: 몇인분? <br>
CKG_DODF_NM: 난이도<br>
CKG_TIME: 요리 시간 <br>


In [10]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/recipe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/recipe


In [ ]:
import pickle
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

file_path = 'Recipe_Info.txt'
data = pd.read_csv(file_path, delimiter='\t', header=None, names=["combined_features"])
#data=data[:100]

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_bert_embeddings(text_list):
    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

data_embeddings = get_bert_embeddings(data['combined_features'].tolist())

def save_model_and_embeddings(model, embeddings, file_path='model_embeddings.pkl'):
    with open(file_path, 'wb') as f:
        pickle.dump((model, embeddings), f)
    print(f"Model and embeddings saved to {file_path}")

save_model_and_embeddings(model, data_embeddings)



Model and embeddings saved to model_embeddings.pkl
Model and embeddings loaded from model_embeddings.pkl
추천 요리:
요리 인덱스: 8, 유사도 점수: 0.49467194080352783


AttributeError: 'Series' object has no attribute 'value'

In [11]:
import pickle
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load model and embeddings from pickle file
def load_model_and_embeddings(file_path='model_embeddings.pkl'):
    with open(file_path, 'rb') as f:
        model, embeddings = pickle.load(f)
    print(f"Model and embeddings loaded from {file_path}")
    return model, embeddings

# Function to recommend recipe based on user input
def recommend_recipe(user_input, model, data_embeddings, tokenizer, file_path='Recipe_Info.txt'):
    # Define get_bert_embeddings function inside recommend_recipe
    def get_bert_embeddings(text_list, tokenizer, model):
        inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

    data = pd.read_csv(file_path, delimiter='\t', header=None, names=["combined_features"])
    user_embedding = get_bert_embeddings([user_input], tokenizer, model)
    sim_scores = cosine_similarity(user_embedding, data_embeddings).flatten()
    sim_indices = sim_scores.argsort()[-5:][::-1]
    recommendations = [(data.iloc[i]['combined_features'], sim_scores[i]) for i in sim_indices]
    return recommendations

# Load the model and embeddings
loaded_model, loaded_embeddings = load_model_and_embeddings('model_embeddings.pkl')

# Initialize BERT tokenizer (necessary for get_bert_embeddings)
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Use the loaded model and embeddings to recommend recipes
user_input = input('뭘 도와줄까: ')
recommendations = recommend_recipe(user_input, loaded_model, loaded_embeddings, tokenizer)
print(recommendations)


Model and embeddings loaded from model_embeddings.pkl
뭘 도와줄까: 고기
[('요리 이름: 찜닭, 추천 횟수: 9, 스크랩 횟수: 98, 요리 방법: 찜, 요리 목적: 손님접대, 주재료의 종류: 닭고기, 요리 종류: 메인반찬, 요리 상세 재료: 닭 중간크기 1마리, 청양고추 5개, 건고추 5개, 몇인분: 4인분, 난이도: 초급, 요리 시간: 60분이내', 0.4725165), ('요리 이름: 고구마보트, 추천 횟수: 0, 스크랩 횟수: 12, 요리 방법: 굽기, 요리 목적: 간식, 주재료의 종류: 채소류, 요리 종류: 디저트, 요리 상세 재료: 고구마 5개, 생크림 3큰술, 버터 2큰술, 몇인분: 3인분, 난이도: 초급, 요리 시간: 30분이내', 0.4610054), ('요리 이름: 오븐웨지감자, 추천 횟수: 8, 스크랩 횟수: 296, 요리 방법: 굽기, 요리 목적: 일상, 주재료의 종류: 채소류, 요리 종류: 디저트, 요리 상세 재료: 감자 중간것 4개 [양념] 올리브유(or카놀라유) 2큰술, 소금 1작은술, 후춧가루 약간, 몇인분: 4인분, 난이도: 초급, 요리 시간: 15분이내', 0.4584295), ('요리 이름: 버팔로윙스, 추천 횟수: 1, 스크랩 횟수: 53, 요리 방법: 굽기, 요리 목적: 술안주, 주재료의 종류: 닭고기, 요리 종류: 양식, 요리 상세 재료: 닭날개 20개, 재움용 우유 1컵 [허니머스타드소스] 머스타드 1큰술, 마요네즈 1큰술, 몇인분: 6인분이상, 난이도: 초급, 요리 시간: 2시간이상', 0.45671767), ('요리 이름: 두부조림, 추천 횟수: 9, 스크랩 횟수: 44, 요리 방법: 조림, 요리 목적: 일상, 주재료의 종류: 콩/견과류, 요리 종류: 밑반찬, 요리 상세 재료: 손두부 1모, 포도씨유, 소금 약간, 몇인분: 3인분, 난이도: 초급, 요리 시간: 15분이내', 0.45652843)]
